In [1]:
import os
import random
import skimage.data
from skimage import io
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
def get_train_data_batch(num, pic_dir, label_dir):
    data = [0,1,2,3,4]
    data = np.array(data)
    one_hots = to_categorical(data)
    
    train_txt = os.path.join(label_dir,"label.txt")
    count = len(open(train_txt,'r').readlines())    
    idx = np.arange(0, count)      
    np.random.shuffle(idx)                
    data_idx = idx[:num]        
    
    labels = []
    images = []
    
    for i in range(num):
        with open(train_txt,'r') as file:
            s = file.readlines()
            img = skimage.data.imread(os.path.join(pic_dir, s[data_idx[i]].split(";")[0]))
            images.append(img)
            label = int(s[data_idx[i]].split(";")[1])
            labels.append(one_hots[label])
    return np.asarray(images),np.asarray(labels)
train_label_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/00"
train_pic_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/00/picture"

num = 4
train_images, train_labels = get_train_data_batch(num,train_pic_dir,train_label_dir)
print(np.shape(train_images))
print(np.shape(train_labels))
print(type(train_labels))


(4, 896, 896, 3)
(4, 5)
<class 'numpy.ndarray'>


In [3]:
def load_test_data(pic_dir,label_dir):
    data = [0,1,2,3,4]
    data = np.array(data)
    one_hots = to_categorical(data)
    
    train_txt = os.path.join(label_dir,"label.txt")
    labels = []
    images = []
    with open(train_txt,'r') as f:
        all_file_name = [x.strip() for x in f.readlines()]
        for fil in all_file_name:
            img = skimage.data.imread(os.path.join(pic_dir,fil.split(";")[0]))
            images.append(img)
            label = int([fil.split(";")[-1]][0])
            labels.append(one_hots[label])
    return np.asarray(images), np.asarray(labels)

test_label_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/01"
test_pic_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/01/picture"

test_images, test_labels = load_test_data(test_pic_dir,test_label_dir)
print(np.shape(test_images))
print(np.shape(test_labels))
print(type(test_labels[0]))

(88, 896, 896, 3)
(88, 5)
<class 'numpy.ndarray'>


In [4]:
def get_test_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

X, Y = get_test_batch(10, test_images, test_labels)
print(Y)
print(np.shape(X))
print(np.shape(Y))

[[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]
(10, 896, 896, 3)
(10, 5)


In [5]:
with tf.name_scope("input"):
    input_image = tf.placeholder("float", [None, 896, 896, 3],name='input_image')
    y_ = tf.placeholder("float", [None, 5],name='y_') 

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

#第0层卷积层
#参数的前两维为卷积核的维度，第三个参数为当前层的深度，第四个为输出到层的深度

W_conv0 = weight_variable([7, 7, 3, 64])
b_conv0 = bias_variable([64])

h_conv0 = tf.nn.relu(tf.nn.conv2d(input_image, W_conv0, strides=[1,2,2,1], padding="SAME") + b_conv0)
h_pool0 = max_pool_2x2(h_conv0)   #输出变成224*224*64 [None,224,224,64]

# #第一层卷积层
# #参数的前两维为卷积核的维度，第三个参数为当前层的深度，第四个为输出到层的深度

W_conv1 = weight_variable([7, 7, 64, 64])
b_conv1 = bias_variable([64])

h_conv1 = tf.nn.relu(conv2d(h_pool0, W_conv1) + b_conv1)
# h_conv1 = tf.nn.relu(tf.nn.conv2d(input_image, W_conv1, strides=[1,2,2,1], padding="SAME") + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)   #输出变成112*112*64   [None,112,112,64]

#第二层卷积层
W_conv2 = weight_variable([3, 3, 64, 192])
b_conv2 = bias_variable([192])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)  #输出变为56*56*192  [4,56,56,192]

#第三层卷积层
W_conv3 = weight_variable([3, 3, 192, 128])
b_conv3 = bias_variable([128])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)   #输出为28*28*128  [4,28,28,192]


#第四层卷积层
W_conv4 = weight_variable([3, 3, 128, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4) + b_conv4)
h_pool4 = max_pool_2x2(h_conv4)   #输出变为14*14*64 [None, 14,14,64]

#密集连接层1
W_fc1 = weight_variable([14*14*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool4, [-1, 14*14*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)   #输出为[None,1024]

keep_prob = tf.placeholder("float")
#relu函数作用是求出max(h_fc1,0)
h_fc1_drop1 = tf.nn.dropout(h_fc1, keep_prob)   #输出为[None, 1024]

#密集连接层2
W_fc2 = weight_variable([1024, 512])
b_fc2 = bias_variable([512])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop1, W_fc2) + b_fc2)   #输出为[None,512]
#relu函数作用是求出max(h_fc1,0)
h_fc1_drop2 = tf.nn.dropout(h_fc2, keep_prob)

#输出层softmax层
W_fc3 = weight_variable([512, 5])
b_fc3 = bias_variable([5])

y_conv = tf.matmul(h_fc1_drop2, W_fc3) + b_fc3

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     batch_x,batch_y = get_train_data_batch(4, train_pic_dir, train_label_dir)
#     print(batch_x.shape)
#     print(sess.run(tf.shape(y_conv),feed_dict={input_image:batch_x, y_:batch_y, keep_prob:1.0}))
    
starter_learning_rate = 0.01
steps_per_decay = 1000
decay_factor = 0.96
 
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(learning_rate = starter_learning_rate,
                                           global_step = global_step,
                                           decay_steps = steps_per_decay,
                                           decay_rate = decay_factor,
                                           staircase = True,#If `True` decay the learning rate at discrete intervals
                                           #staircase = False,change learning rate at every step
                                           )

# cross_entropy = tf.reduce_sum(tf.square(y_-y_conv)) 
# cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
cross_entropy = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv))
tf.summary.scalar("cross_entropy",cross_entropy)

# train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar("accuracy",accuracy)
merged = tf.summary.merge_all()

batch_size = 4
                
checkpointsPath = "./896_classs_checkpoints/"
reload = True

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #保存训练好的模型一遍下次使用
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter("logs_dir/", sess.graph)
    
    if not os.path.exists(checkpointsPath):
        os.mkdir(checkpointsPath)
    
    if reload:
        checkPoint = tf.train.get_checkpoint_state(checkpointsPath)
        if checkPoint and checkPoint.model_checkpoint_path:
            saver.restore(sess,checkPoint.model_checkpoint_path)
            print("restored %s" % checkPoint.model_checkpoint_path)
        else:
            print("no checkpoint found!")
        
    for i in tqdm(range(20000)):
        batch_x,batch_y = get_train_data_batch(batch_size, train_pic_dir, train_label_dir)
        summary,_ = sess.run([merged,train_step],feed_dict={input_image:batch_x, y_:batch_y, 
                                                            keep_prob:0.8,global_step:i})
        writer.add_summary(summary, i)
        if (i % 100) == 0 & (i != 0):
            print("save model")
            saver.save(sess, os.path.join(checkpointsPath,"saved_net.ckpt"), global_step=i)
            print("loss is:",sess.run(cross_entropy,feed_dict={input_image:batch_x, y_:batch_y, keep_prob:1.0}))
            test_batch_x,test_batch_y = get_test_batch(30, test_images, test_labels)
            print("learning_rate is:",sess.run(learning_rate,feed_dict = {global_step:i}))
            print("test accuraracy: %.5f" %accuracy.eval(feed_dict={input_image:test_batch_x,
                                                                    y_:test_batch_y,
                                                                    keep_prob:1.0}))
    print("done!")


  0%|          | 0/20000 [00:00<?, ?it/s]

no checkpoint found!
save model
loss is: 192924.9
learning_rate is: 0.01


  0%|          | 1/20000 [00:10<57:41:14, 10.38s/it]

test accuraracy: 0.30000


  0%|          | 100/20000 [00:37<1:24:25,  3.93it/s]

...

save model
loss is: 0.0
learning_rate is: 0.0046041897


100%|█████████▉| 19901/20000 [1:17:33<00:42,  2.32it/s]

test accuraracy: 0.96667


100%|██████████| 20000/20000 [1:17:55<00:00,  4.57it/s]

done!
